!pip install git+https://gitlab.com/pavanello-research-group/qmlearn.git

!pip install ase

!wget https://gitlab.com/pavanello-research-group/qmlearn/-/raw/main/examples/examples/h2o/h2o_md_300_QML_set.hdf5

## Using a QMModel: Performing Geometry Optimization

In [1]:
from ase.build import molecule
from ase.optimize import LBFGS

In [2]:
from qmlearn.api.api4ase import QMLCalculator
from qmlearn.drivers.mol import QMMol
from qmlearn.io.model import db2qmmodel

##### Get Energies using Delta learning by altering the initial geometry 

1) Open a database that already defined the Linear Regression model using `db2qmmodel`

In [3]:
qmmodel = db2qmmodel('h2o_vib_QML_set.hdf5')

Guess DB names : {'qmmol': 'rks/qmmol', 'atoms': 'rks/train_atoms_36', 'properties': 'rks/train_props_36'}
Guess mmodels: {'gamma': KernelRidge(alpha=0.1), 'd_gamma': LinearRegression(), 'd_energy': LinearRegression(), 'd_forces': LinearRegression()}


2) Define second learning features `energy` and `forces` 

3) Define the Calculator, based on `second_learn` models. 

In [4]:
second_learn = {
        'energy' : 'd_energy',
        'forces' : 'd_forces',
        }
calc = QMLCalculator(qmmodel = qmmodel, second_learn = second_learn, method = 'gamma')

4) Alter the atom_test positions by 0.2 and perform an optimization using `LBFGS` method.

In [5]:
atoms_test=molecule('H2O')
atoms_test.positions[0,0] += 0.2

In [6]:
atoms = atoms_test.copy()
atoms.calc = calc

In [7]:
LBFGS(atoms, trajectory = 'opt.traj').run(fmax=0.01)
atoms.positions

       Step     Time          Energy         fmax
LBFGS:    0 14:35:16    -2070.590297        1.2629
LBFGS:    1 14:35:16    -2070.613641        0.5462
LBFGS:    2 14:35:16    -2070.613731        0.5392
LBFGS:    3 14:35:16    -2070.618353        0.0477
LBFGS:    4 14:35:17    -2070.618367        0.0338
LBFGS:    5 14:35:17    -2070.618416        0.0159
LBFGS:    6 14:35:17    -2070.618417        0.0050


array([[ 1.99131068e-01, -1.85111165e-05,  9.48952203e-02],
       [ 4.35154210e-04,  7.65802993e-01, -4.64833266e-01],
       [ 4.33783194e-04, -7.65784499e-01, -4.64893925e-01]])

##### Get Energies using PySCF engine

1) Define the basis set, level of theory, exchange correlation functional and the total charge of your system

In [8]:
basis = 'cc-pvTZ'
xc = 'lda,vwn_rpa'
method = 'rks'
charge = 0

2) Create atoms `atoms` copy, and the `refqmmol` object to initialize the engine.

3) Initialize the calculator using `QMLCalculator` class.

In [9]:
atoms = atoms_test.copy()
refqmmol = QMMol(atoms = atoms, method = method, basis=basis, xc = xc, charge=charge)
atoms.calc = QMLCalculator(qmmodel = refqmmol, method = 'engine')

4) Run the geometry optimization

In [10]:
LBFGS(atoms, trajectory = 'opt.traj').run(fmax=0.01)
atoms.positions

       Step     Time          Energy         fmax
LBFGS:    0 14:35:22    -2070.596525        1.3259
LBFGS:    1 14:35:25    -2070.616301        0.2563
LBFGS:    2 14:35:28    -2070.617393        0.1126
LBFGS:    3 14:35:30    -2070.618015        0.1030
LBFGS:    4 14:35:32    -2070.618396        0.0495
LBFGS:    5 14:35:35    -2070.618425        0.0106
LBFGS:    6 14:35:38    -2070.618427        0.0005


array([[ 1.92577545e-01, -2.07643072e-16,  9.72371798e-02],
       [ 3.71122733e-03,  7.65804799e-01, -4.66034590e-01],
       [ 3.71122733e-03, -7.65804799e-01, -4.66034590e-01]])

##### Get Energies using Delta learning  without altering the geometry

1) Open a model in a database and keep it in `qmmodel`.

2) Create atoms `atoms` copy.

3) Initialize the calculator using `QMLCalculator` class, based on the define `second_learn` models.

In [11]:
qmmodel = db2qmmodel('./h2o_md_300_QML_set.hdf5')
atoms = atoms_test.copy()
atoms.calc = QMLCalculator(qmmodel = qmmodel, second_learn = second_learn, method = 'gamma')

Guess DB names : {'qmmol': 'rks/qmmol', 'atoms': 'rks/train_atoms_30', 'properties': 'rks/train_props_30'}
Guess mmodels: {'gamma': KernelRidge(alpha=0.1), 'd_gamma': LinearRegression(), 'd_energy': LinearRegression(), 'd_forces': LinearRegression()}


4) Run the geometry optimization

In [12]:
LBFGS(atoms, trajectory = 'opt.traj').run(fmax=0.01)
atoms.positions

       Step     Time          Energy         fmax
LBFGS:    0 14:35:42    -2070.596523        1.3260
LBFGS:    1 14:35:42    -2070.616324        0.2564
LBFGS:    2 14:35:42    -2070.617414        0.1126
LBFGS:    3 14:35:42    -2070.618033        0.1029
LBFGS:    4 14:35:43    -2070.618410        0.0495
LBFGS:    5 14:35:43    -2070.618441        0.0106
LBFGS:    6 14:35:43    -2070.618442        0.0015


array([[ 1.92610233e-01,  1.21723284e-05,  9.72290782e-02],
       [ 3.69637070e-03,  7.65793860e-01, -4.66026104e-01],
       [ 3.69340087e-03, -7.65806039e-01, -4.66034959e-01]])